## Average tempreture of 5 stations in Harz

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import openmeteo_requests
import requests_cache
from retry_requests import retry
from sklearn.metrics import mean_squared_error

In [3]:
# resuable code

cache_session = requests_cache.CachedSession('.cache', expire_after = 3600)
retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
openmeteo = openmeteo_requests.Client(session = retry_session)

# Function for frequent use for houlrly data
#https://open-meteo.com/en/docs/historical-weather-api#start_date=1950-01-01&end_date=2023-12-31&hourly=&daily=temperature_2m_mean&timezone=Europe%2FBerlin&models=era5_seamless
def daily_open_meteo_data(start_date:str, end_date:str, lat:list, long:list, variable):
	url = "https://archive-api.open-meteo.com/v1/archive"
	params = {
	"latitude": lat,
	"longitude": long,
	"start_date":start_date,
	"end_date": end_date,
	"timezone": "Europe/Berlin",
	"daily": variable,
	"models": ["era5_seamless"]
	}
	responses = openmeteo.weather_api(url, params=params)
	response = responses[0]
	# Process daily data. The order of variables needs to be the same as requested.
	daily = response.Daily()
	daily_sum = daily.Variables(0).ValuesAsNumpy()

	daily_data = {"date": pd.date_range(
	start = pd.to_datetime(daily.Time(), unit = "s", utc = True),
	end = pd.to_datetime(daily.TimeEnd(), unit = "s", utc = True),
	freq = pd.Timedelta(seconds = daily.Interval()),
	inclusive = "left"
	)}
	daily_data["open_meteo"] = daily_sum

	daily_dataframe = pd.DataFrame(data = daily_data)
	if daily_dataframe.empty:
		return print("The daily limit reached")
	return daily_dataframe

def calculate_accuracy(original_data, predicted_data):
	mbe = np.mean(predicted_data - original_data)
	rmse = np.sqrt(mean_squared_error(original_data, predicted_data))
	correlation = np.corrcoef(original_data, predicted_data)[0, 1]

	# Print results
	print(f"Mean Bias Error (MBE): {mbe}")
	print(f"Root Mean Square Error (RMSE): {rmse}")
	print(f"Correlation: {correlation}")
	

# Function for frequent use for houlrly data
#https://open-meteo.com/en/docs/historical-weather-api#start_date=1950-01-01&end_date=2023-12-31&hourly=&daily=temperature_2m_mean&timezone=Europe%2FBerlin&models=era5_seamless
def Hourly_open_meteo_data(start_date:str, end_date:str, lat:list, long:list, variable):
	url = "https://archive-api.open-meteo.com/v1/archive"
	params = {
	"latitude": lat,
	"longitude": long,
	"start_date":start_date,
	"end_date": end_date,
	"timezone": "Europe/Berlin",
	"hourly": variable,
	"models": ["era5_seamless"]
	}
	responses = openmeteo.weather_api(url, params=params)
	response = responses[0]
	# Process daily data. The order of variables needs to be the same as requested.
	hourly = response.Hourly()
	hourly_snow_depth = hourly.Variables(0).ValuesAsNumpy()

	hourly_data = {"date": pd.date_range(
		start = pd.to_datetime(hourly.Time(), unit = "s", utc = True),
		end = pd.to_datetime(hourly.TimeEnd(), unit = "s", utc = True),
		freq = pd.Timedelta(seconds = hourly.Interval()),
		inclusive = "left"
	)}
	hourly_data["open_meteo"] = hourly_snow_depth

	hourly_dataframe = pd.DataFrame(data = hourly_data)

	if hourly_dataframe.empty:
		return print("The daily limit reached")
	return hourly_dataframe



# plots
def plot_my_bargraph(length:int,breath:int,xaxis,yaxis,title:str,xlabel:str,ylabel:str, widthv=0.5, colorv='blue',xtick = False):
    plt.figure(figsize=(length, breath))
    plt.bar(xaxis, yaxis, width = widthv , color=colorv)
    plt.title(title)
    plt.xlabel(xlabel)
    plt.ylabel(ylabel)
    plt.grid(axis='y')
    if xtick == True:
        plt.xticks(xaxis,rotation=45,fontsize=10)
    plt.tight_layout()
    plt.show()



def plot_my_bargraph_withnumbers(length:int,breath:int,xaxis,yaxis,title:str,xlabel:str,ylabel:str, widthv=0.5, colorv='blue'):
    plt.figure(figsize=(length, breath))
    plt.bar(xaxis, yaxis, width = widthv , color=colorv)
    plt.title(title)
    plt.xlabel(xlabel)
    plt.ylabel(ylabel)
    plt.grid(axis='y')
    for i, value in enumerate(yaxis):
        plt.text(xaxis[i], value + 1, str(value), ha='center', fontsize=9)
    plt.tight_layout()
    plt.xticks(xaxis,rotation=45,fontsize=10)
    plt.show()

### Creating the function for averaging the data for 5 stations

In [4]:
def avg_5_station(variable):
    main_df = pd.DataFrame()
    avg_main_df = pd.DataFrame()  
    stations = [(51.95,10.23),(51.73,10.56),(51.67,10.78),(51.5,11.1),(51.5,11.3)] #five stations
    for i, station in enumerate(stations):
        data = daily_open_meteo_data("1950-01-01", "2023-12-31",station[0], station[1],variable)
        main_df['column'+ str(i)] = data["open_meteo"]
    avg_main_df["open_meteo"] = main_df.mean(axis=1)
    return avg_main_df

In [5]:
precipitation_data = avg_5_station("precipitation_sum")
precipitation_data

OpenMeteoRequestsError: {'reason': 'Minutely API request limit exceeded. Please try again in one minute.', 'error': True}

### Due to continous data calling we are gettting error so we are doing manually file creation and export.

In [ ]:
#avg_5_main_df_preci
#avg_5_main_df_Et0

In [6]:
open_meto_1950_2023_precipitation_1 = daily_open_meteo_data("1950-01-01", "2023-12-31",[51.95], [10.23],"precipitation_sum")

In [7]:
open_meto_1950_2023_precipitation_2 = daily_open_meteo_data("1950-01-01", "2023-12-31",[51.73], [10.56],"precipitation_sum")
open_meto_1950_2023_precipitation_1['precipitation_2'] = open_meto_1950_2023_precipitation_2["open_meteo"]

In [8]:
open_meto_1950_2023_precipitation_3 = daily_open_meteo_data("1950-01-01", "2023-12-31",[51.67], [10.78],"precipitation_sum")
open_meto_1950_2023_precipitation_1['precipitation_3'] = open_meto_1950_2023_precipitation_3["open_meteo"]

In [9]:
open_meto_1950_2023_precipitation_4 = daily_open_meteo_data("1950-01-01", "2023-12-31",[51.5], [11.1],"precipitation_sum")
open_meto_1950_2023_precipitation_1['precipitation_4'] = open_meto_1950_2023_precipitation_4["open_meteo"]

In [10]:
open_meto_1950_2023_precipitation_5 = daily_open_meteo_data("1950-01-01", "2023-12-31",[51.5], [11.3],"precipitation_sum")
open_meto_1950_2023_precipitation_1['precipitation_5'] = open_meto_1950_2023_precipitation_5["open_meteo"]

In [11]:
open_meto_1950_2023_precipitation_1['average_precipitation'] = open_meto_1950_2023_precipitation_1[["open_meteo",'precipitation_2','precipitation_3','precipitation_4','precipitation_5']].mean(axis=1)
open_meto_1950_2023_precipitation_1

,date,open_meteo,precipitation_2,precipitation_3,precipitation_4,precipitation_5,average_precipitation
0,1949-12-31 23:00:00+00:00,0.000000,0.000000,0.000000,0.0,0.000000,0.000000
1,1950-01-01 23:00:00+00:00,12.000001,11.900001,11.900001,9.1,9.600001,10.900001
2,1950-01-02 23:00:00+00:00,3.500000,3.500000,3.500000,0.5,0.200000,2.240000
3,1950-01-03 23:00:00+00:00,2.400000,2.100000,2.100000,1.0,0.800000,1.680000
4,1950-01-04 23:00:00+00:00,5.500000,6.200000,6.200000,4.5,3.700000,5.220000
...,...,...,...,...,...,...,...
27023,2023-12-26 23:00:00+00:00,0.800000,0.300000,0.300000,0.1,0.000000,0.300000
27024,2023-12-27 23:00:00+00:00,0.000000,0.100000,0.100000,0.0,0.000000,0.040000
27025,2023-12-28 23:00:00+00:00,8.599999,9.099999,9.099999,3.6,2.900000,6.660000
27026,2023-12-29 23:00:00+00:00,2.000000,3.600000,3.600000,1.7,0.700000,2.320000


In [12]:
avg_5_main_df_preci = open_meto_1950_2023_precipitation_1.copy()
avg_5_main_df_preci.drop(['open_meteo','precipitation_2','precipitation_3','precipitation_4','precipitation_5'],axis=1,inplace = True)
avg_5_main_df_preci

,date,average_precipitation
0,1949-12-31 23:00:00+00:00,0.000000
1,1950-01-01 23:00:00+00:00,10.900001
2,1950-01-02 23:00:00+00:00,2.240000
3,1950-01-03 23:00:00+00:00,1.680000
4,1950-01-04 23:00:00+00:00,5.220000
...,...,...
27023,2023-12-26 23:00:00+00:00,0.300000
27024,2023-12-27 23:00:00+00:00,0.040000
27025,2023-12-28 23:00:00+00:00,6.660000
27026,2023-12-29 23:00:00+00:00,2.320000


In [ ]:
avg_5_main_df_preci['date'] = avg_5_main_df_preci['date'].dt.tz_localize(None)
avg_5_main_df_preci.set_index(['date'],inplace=True)

In [31]:
file_name = 'precipitation_avergae.xlsx'


#avg_5_main_df_preci
avg_5_main_df_preci.to_excel(file_name)

### Data for Et0 

In [15]:
open_meto_1950_2023_et0_1 = daily_open_meteo_data("1950-01-01", "2023-12-31",[51.95], [10.23],"et0_fao_evapotranspiration")

In [16]:
open_meto_1950_2023_et0_2 = daily_open_meteo_data("1950-01-01", "2023-12-31",[51.73], [10.56],"et0_fao_evapotranspiration")
open_meto_1950_2023_et0_1['et0_2'] = open_meto_1950_2023_et0_2["open_meteo"]

In [17]:
open_meto_1950_2023_et0_3 = daily_open_meteo_data("1950-01-01", "2023-12-31",[51.67], [10.78],"et0_fao_evapotranspiration")
open_meto_1950_2023_et0_1['et0_3'] = open_meto_1950_2023_et0_3["open_meteo"]

In [18]:
open_meto_1950_2023_et0_4 = daily_open_meteo_data("1950-01-01", "2023-12-31",[51.5], [11.1],"et0_fao_evapotranspiration")
open_meto_1950_2023_et0_1['et0_4'] = open_meto_1950_2023_et0_4["open_meteo"]

In [22]:
open_meto_1950_2023_et0_5 = daily_open_meteo_data("1950-01-01", "2023-12-31",[51.5], [11.3],"et0_fao_evapotranspiration")
open_meto_1950_2023_et0_1['et0_5'] = open_meto_1950_2023_et0_5["open_meteo"]

In [23]:
open_meto_1950_2023_et0_1['average_et0'] = open_meto_1950_2023_et0_1[["open_meteo",'et0_2','et0_3','et0_4','et0_5']].mean(axis=1)
open_meto_1950_2023_et0_1

,date,open_meteo,et0_2,et0_3,et0_4,et0_5,average_et0
0,1949-12-31 23:00:00+00:00,0.503220,0.433384,0.430267,0.442433,0.468725,0.455606
1,1950-01-01 23:00:00+00:00,0.660590,0.689421,0.741532,0.938922,0.941560,0.794405
2,1950-01-02 23:00:00+00:00,0.213881,0.162375,0.178739,0.219877,0.254170,0.205808
3,1950-01-03 23:00:00+00:00,0.219946,0.224397,0.232650,0.272752,0.277204,0.245390
4,1950-01-04 23:00:00+00:00,0.150908,0.144171,0.150264,0.187341,0.220099,0.170557
...,...,...,...,...,...,...,...
27023,2023-12-26 23:00:00+00:00,0.336182,0.284533,0.291716,0.371627,0.414460,0.339704
27024,2023-12-27 23:00:00+00:00,1.310526,1.029314,1.073771,1.243521,1.310785,1.193583
27025,2023-12-28 23:00:00+00:00,0.794533,0.584567,0.628552,0.883992,0.973888,0.773106
27026,2023-12-29 23:00:00+00:00,0.578180,0.438660,0.464509,0.602797,0.663943,0.549618


In [24]:
avg_5_main_df_Et0 = open_meto_1950_2023_et0_1.copy()
avg_5_main_df_Et0.drop(["open_meteo",'et0_2','et0_3','et0_4','et0_5'],axis=1,inplace = True)
avg_5_main_df_Et0

,date,average_et0
0,1949-12-31 23:00:00+00:00,0.455606
1,1950-01-01 23:00:00+00:00,0.794405
2,1950-01-02 23:00:00+00:00,0.205808
3,1950-01-03 23:00:00+00:00,0.245390
4,1950-01-04 23:00:00+00:00,0.170557
...,...,...
27023,2023-12-26 23:00:00+00:00,0.339704
27024,2023-12-27 23:00:00+00:00,1.193583
27025,2023-12-28 23:00:00+00:00,0.773106
27026,2023-12-29 23:00:00+00:00,0.549618


In [34]:
avg_5_main_df_Et0['date'] = avg_5_main_df_Et0['date'].dt.tz_localize(None)
avg_5_main_df_Et0.set_index(['date'],inplace=True)

In [35]:
file_name = 'Et0_average.xlsx'
#vg_5_main_df_Et0['date'] = avg_5_main_df_Et0['date'].dt.tz_localize(None)
avg_5_main_df_Et0.to_excel(file_name)